# Set Up and Load OpenAQ data into BigQuery

-  Reads local CSV (bulk OpenAQ dataset).

-  Creates & populates the BigQuery table from scratch.

In [11]:
# 📌 1. Setup environment & imports

import os
from google.cloud import bigquery
import pandas as pd
from dotenv import load_dotenv

# Load environment variables (e.g., GOOGLE_APPLICATION_CREDENTIALS)
load_dotenv()

# Confirm key path
print("Google Credentials Path:", os.getenv("GOOGLE_APPLICATION_CREDENTIALS"))

# Initialize BigQuery client
client = bigquery.Client()

Google Credentials Path: C:\\Users\\camer\\.gcp_keys\\openaq_data_loader.json


In [3]:
# 📌 2. Define your dataset and table names

PROJECT_ID = client.project  # or hardcode your project id here
DATASET_ID = "openaq_ca"
TABLE_ID = "pm25_hourly_ca"
FULL_TABLE_ID = f"{PROJECT_ID}.{DATASET_ID}.{TABLE_ID}"

print(f"BigQuery Dataset: {DATASET_ID}")
print(f"BigQuery Table: {TABLE_ID}")

BigQuery Dataset: openaq_ca
BigQuery Table: pm25_hourly_ca


In [3]:
# 📌 3. Create the dataset if it doesn’t exist

def create_dataset(dataset_id):
    dataset_ref = client.dataset(dataset_id)
    try:
        dataset = client.get_dataset(dataset_ref)
        print(f"Dataset {dataset_id} already exists.")
    except Exception:
        dataset = bigquery.Dataset(dataset_ref)
        dataset.location = "US"  # or your preferred region
        dataset = client.create_dataset(dataset)
        print(f"Created dataset {dataset_id}")

create_dataset(DATASET_ID)

Created dataset openaq_ca


In [15]:
# 📌 4. Load local CSV data to BigQuery table (replace path as needed)

CSV_PATH = "../data/openaq_hourly_ca_20160101_to_20250731.csv"

def load_csv_to_bq(csv_path, dataset_id, table_id):
    table_ref = client.dataset(dataset_id).table(table_id)

    job_config = bigquery.LoadJobConfig(
        autodetect=True,       # Auto-detect schema from data
        write_disposition="WRITE_TRUNCATE",  # Overwrite table if exists
        source_format=bigquery.SourceFormat.CSV,
        skip_leading_rows=1   # Skip CSV header
    )

    with open(csv_path, "rb") as source_file:
        load_job = client.load_table_from_file(
            source_file,
            table_ref,
            job_config=job_config,
        )

    load_job.result()  # Wait for job to complete
    print(f"Loaded {load_job.output_rows} rows into {dataset_id}.{table_id}.")

load_csv_to_bq(CSV_PATH, DATASET_ID, TABLE_ID)

Loaded 851736 rows into openaq_ca.pm25_hourly_ca.


In [16]:
# 📌 5. Verify loaded data with a simple query

query = f"""
SELECT sensor_id, COUNT(*) as count_records
FROM `{FULL_TABLE_ID}`
GROUP BY sensor_id
ORDER BY count_records DESC
LIMIT 10
"""

query_job = client.query(query)
results = query_job.result()

print("Top 10 sensors by record count:")
for row in results:
    print(f"Sensor ID: {row.sensor_id}, Records: {row.count_records}")

Top 10 sensors by record count:
Sensor ID: 1561, Records: 18986
Sensor ID: 1646, Records: 18622
Sensor ID: 1618, Records: 18232
Sensor ID: 1591, Records: 18130
Sensor ID: 1595, Records: 18000
Sensor ID: 1502, Records: 18000
Sensor ID: 1624, Records: 18000
Sensor ID: 1630, Records: 18000
Sensor ID: 1639, Records: 18000
Sensor ID: 1598, Records: 18000


In [17]:
# show table

query = f"""
SELECT *
FROM `{FULL_TABLE_ID}`
LIMIT 10
"""

query_job = client.query(query)
results = query_job.result()

for row in results:
    print(row)

Row((350, 9.0, 'µg/m³', 'pm25', datetime.datetime(2016, 3, 6, 19, 0, tzinfo=datetime.timezone.utc), datetime.datetime(2016, 3, 6, 19, 0, tzinfo=datetime.timezone.utc), datetime.datetime(2016, 3, 6, 20, 0, tzinfo=datetime.timezone.utc), datetime.datetime(2016, 3, 6, 20, 0, tzinfo=datetime.timezone.utc), 100.0), {'sensor_id': 0, 'value': 1, 'units': 2, 'pollutant': 3, 'datetime_from_utc': 4, 'datetime_from_local': 5, 'datetime_to_utc': 6, 'datetime_to_local': 7, 'coverage_pct': 8})
Row((401, 2.3, 'µg/m³', 'pm25', datetime.datetime(2016, 3, 6, 19, 0, tzinfo=datetime.timezone.utc), datetime.datetime(2016, 3, 6, 19, 0, tzinfo=datetime.timezone.utc), datetime.datetime(2016, 3, 6, 20, 0, tzinfo=datetime.timezone.utc), datetime.datetime(2016, 3, 6, 20, 0, tzinfo=datetime.timezone.utc), 100.0), {'sensor_id': 0, 'value': 1, 'units': 2, 'pollutant': 3, 'datetime_from_utc': 4, 'datetime_from_local': 5, 'datetime_to_utc': 6, 'datetime_to_local': 7, 'coverage_pct': 8})
Row((537, 2.1, 'µg/m³', 'pm25'

In [18]:
df = query_job.to_dataframe()
print(df.head())

c:\Users\camer\professional\data science\projects\openaq_pipeline_project\venv\Lib\site-packages\google\cloud\bigquery\table.py:1965: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


   sensor_id  value  units pollutant         datetime_from_utc  \
0        350    9.0  µg/m³      pm25 2016-03-06 19:00:00+00:00   
1        401    2.3  µg/m³      pm25 2016-03-06 19:00:00+00:00   
2        537    2.1  µg/m³      pm25 2016-03-06 19:00:00+00:00   
3       3050    0.5  µg/m³      pm25 2016-03-06 19:00:00+00:00   
4       1556    7.0  µg/m³      pm25 2016-03-06 19:00:00+00:00   

        datetime_from_local           datetime_to_utc  \
0 2016-03-06 19:00:00+00:00 2016-03-06 20:00:00+00:00   
1 2016-03-06 19:00:00+00:00 2016-03-06 20:00:00+00:00   
2 2016-03-06 19:00:00+00:00 2016-03-06 20:00:00+00:00   
3 2016-03-06 19:00:00+00:00 2016-03-06 20:00:00+00:00   
4 2016-03-06 19:00:00+00:00 2016-03-06 20:00:00+00:00   

          datetime_to_local  coverage_pct  
0 2016-03-06 20:00:00+00:00         100.0  
1 2016-03-06 20:00:00+00:00         100.0  
2 2016-03-06 20:00:00+00:00         100.0  
3 2016-03-06 20:00:00+00:00         100.0  
4 2016-03-06 20:00:00+00:00         100.

In [ ]:
# # Clear the entire table

# # SQL to truncate the table
# truncate_sql = f"TRUNCATE TABLE `{FULL_TABLE_ID}`"

# # Run the query
# query_job = client.query(truncate_sql)
# query_job.result()  # Waits for the query to finish

# print(f"Table {FULL_TABLE_ID} truncated successfully.")


Table openaq-data-pipeline-468404.openaq_ca.pm25_hourly_ca truncated successfully.
